## Import packages

In [2]:
import lxml.html
from bs4 import BeautifulSoup
import requests
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
import string
from nltk.stem import PorterStemmer 
import json
import csv
import pandas as pd
import re
from collections import defaultdict
import numpy as np
import math
import sys
import io
import time

# Some variables that we are going to use throughout the code
d = 20000    #number of documents to analyse
ps = PorterStemmer()  

# 1) Data collection.

First we need to get the files

In [ ]:
list_u=[]
soup = BeautifulSoup(response.text, "lxml")
for link in soup.find_all('a'):
    list_u.append(link.get('href'))

In [ ]:
path = 'E:/DataScience/ADM/Data/'


Then we download the files and save them as url files 

In [ ]:
i=20000
for link in list_u:
        response1 = requests.get(link)
        with io.open(''.join([path, 'article_', str(i), '.html']), 'w', encoding="utf-8") as f:
            try:
                txt=response1.text
            except:
                time.sleep(2*60)
                txt=response1.text
            f.write(txt)
        i+=1

Here we read the url files to extract the info we need

In [ ]:
def collector():
    for i in range(0,20000):
        filename='E:/DataScience/ADM/Data/article_'+str(i)+'.html'
        with io.open(filename,'r', encoding="utf-8") as f:
            s1=f.read()
            soup = BeautifulSoup(s1, 'html')
#extract the title of the page
            try:
                t=soup.title.text.strip('- Wikipedia')
            except:
                t="NA"
#find the "h2" , "ul" tags to find the intro and plot
            intro=""
            try:
                start1 = soup.find('table')
                for elem in start1.next_siblings:
                    if elem.name == 'h2':
                    break
                if elem.name != 'p':
                    continue
                intro1=elem.text        
                intro=intro+intro1
                intro=intro.strip('\n')
        except:
            intro="NA"
        plot=""    
        try:
            start2 = soup.find('div',id='toc')
            
            for el in start2.next_siblings:
                if el.name == 'ul':
                    break
                if el.name == 'p':
                    plot1=el.text
                    plot=plot+plot1
                    plot=plot.strip("\n")  
        except:
            plot='NA'
#find the infobox by its class
        tab=soup.find('table',class_='infobox vevent')
        director = ''
        producer=''
        writer=''
        starring=''
        music=''
        release_date=''
        runtime=''
        country=''
        language=''
        budget=''
#try to find info we need form info box by the related tags
        dic={}
        try:
            for r in tab.find_all('tr'):
                td = r.find("td")
                if td != None:
                    s = td
                    p = r.find("th")
                    if p!=None:
                        dic[p.text.strip("\n")] = s.text.strip("\n")
            try:
                director = dic['Directed by']
            except:
                dic['Directed by']="NA"
            try:
                producer = dic['Produced by']
            except:
                dic['Produced by']="NA"
            try:
                writer=dic['Written by']
            except:
                dic['Written by']="NA"
            try:
                starring=dic['Starring']
            except:
                dic['Starring']="NA"
            try:
                music=dic['Music by']
            except:
                dic['Music by']="NA"
            try:
                release_date=dic['Release date']
            except:
                dic['Release date']="NA"
            try:
                runtime=dic['Running time']
            except:
                dic['Running time']="NA"
            try:
                country=dic['Country']
            except:
                dic['Country']="NA"
            try:
                language=dic['Language']
            except:
                dic['Language']="NA"
            try:
                budget=dic['Budget']
            except:
                dic["Budget"] = "NA"
#append all the information of a document to a list to create the tsv files
            list_t=[]
            list_t.append(t)
            list_t.append(intro)
            list_t.append(plot)
            list_t.append(director)
            list_t.append(producer)
            list_t.append(writer)
            list_t.append(starring)
            list_t.append(music)
            list_t.append(release_date)
            list_t.append(runtime)
            list_t.append(country)
            list_t.append(language)
            list_t.append(budget)
#write the tsv file of each page
            tsv_file= 'article_'+str(i)+'.tsv'
            with open(tsv_file,'wt') as write_tsv:
                tsv_writer = csv.writer(write_tsv, delimiter='\t')
                tsv_writer.writerow(list_t)
        except:
            continue

                     

### 2.1) Preprocessing.

In [ ]:
invdict = defaultdict(list)
voc = {}
count = 0
for j in range(d):
    #handle error if file is not present
    try:
        tsvfile = open(r"C:\Users\39335\Desktop\GitHub\HW3\tsv\article_"+str(j)+".tsv", "r")
    except:
        continue
    
    reader = tsvfile.readlines()
    text = []
    
    # Transform the paragraph in a list with all the words
    for row in reader:
        text += row.split()

    
    sw = set(stopwords.words('english'))
    ps = PorterStemmer()    
    # Preprocess the words
    
    for k in range(len(text)):
        # Remove stop words
        text[k] = text[k].lower()
        if text[k] in sw:
            text[k] = ""

        # Remove punctuation
        table = str.maketrans(dict.fromkeys(string.punctuation))
        text[k] = text[k].translate(table)

        # Stemming
        text[k] = ps.stem(text[k])
        
    
    # We create a vocabulary that pairs each word to an ID.
    # We make a inverted index where the key is the ID of the word and the value is a list of the ID
    # of the documents that word is present
    
    # Remove empty elements
    text[:] = [x for x in text if x != ""]
    
    for word in text:
        
        # Create vocabulary
        if not word in voc:
            voc[word] = count
            count += 1
        
        # Create inverted dictionary
        if j in invdict[voc[word]]:
            continue
        invdict[voc[word]].append(j)
        
            
    tsvfile.close()

# I now save the results in txt files so I can use them to search for the query leater.
with open("Vocabulary.txt", "w") as f:
    f.write(json.dumps(voc))
with open("InvertedIndex.txt", "w") as f:
    f.write(json.dumps(invdict))

### 2.1.2) Execute the query. 

First of all we need to load the vocabulary and inverted index we already build.

In [13]:
# Open the file with the vocabulary and the inverted dictionary
with open(r'C:\Users\39335\Downloads\Vocabulary.txt', 'r') as v:
    vocab = json.load(v)
with open(r'C:\Users\39335\Downloads\InvertedIndex.txt', 'r') as i:
    inverted = json.load(i)

Now we find all the documents that have all the words in the query. The search engine will be a function that takes as input the query and outputs a dataframe with the information of the movies that contains all the words in the query. If we need the ID of the documents (for example in ex 2.2.2) the second argument should be ID=True.

In [14]:
def SearchEngine1(q, ID = False):
    query = []
    # We need to stem the word of the query
    
    for word in q:
        query.append(ps.stem(word))

    querydict = {}
    for word in query:
        if word in vocab:
            querydict[word] = vocab[word]
        else:
            print("The word %s was not found" % word)

    l = []

    for i in inverted:
        for ele in query:
            if i == str(querydict[ele]):
                l.append(inverted[i])

    # I want to find the intersection between the lists of documents ID that contain the query
    # (the documents that contains all the words we want)
    inter = set(l[0])
    for i in range(1, len(l)):
        inter = inter.intersection(set(l[i]))
    if len(inter)==0:
        print("No documents satisfy all the elements in the query.")
        sys.exit()

    # Inter is a set with the ID of the documents we need, now we open the html for each and extract the info we need
    # I want to make 3 lists with title, intro and url for each movie that satisfies the query and then make a dataframe
    title = []
    introduction = []
    url = []

    for doc in inter:
        file = open(r'C:\Users\39335\Desktop\HW3\Data\ article_'+str(doc)+'.html', 'r', encoding="utf-8")
        s1=file.read()
        soup = BeautifulSoup(s1, 'html')
        try:
            t=soup.title.text.strip('- Wikipedia')
        except:
            t="NA"
        title.append(t)

        intro=""
        try:
            start1 = soup.find('table')
            for elem in start1.next_siblings:
                if elem.name == 'h2':
                    break
                if elem.name != 'p':
                    continue
                intro1=elem.text        
                intro=intro+intro1
                intro=intro.strip('\n')
        except:
            intro="NA"
        introduction.append(intro)
        u = "https://en.wikipedia.org/wiki/"+t
        u = u.replace(" ", "_")
        url.append(u)
        

    # Make the data frame
    if ID == True:
        result = np.transpose(pd.DataFrame(np.array([title, introduction, url, list(inter)])))
        result.columns = ["Title","Intro", "Wikipedia Url", "Document ID"]
    else:
        result = np.transpose(pd.DataFrame(np.array([title, introduction, url])))
        result.columns = ["Title","Intro", "Wikipedia Url"]        
    return (result)

### 2.2) Conjunctive query & Ranking score

We want to sort the documents we already obtained by their similarity with the given query. We start by defining  the function that returns a list that contains the tfIdf of a given document for each term term of the query. 

In [15]:
# Calculate the tfIdf s for all the terms in a document
def tfIdf(terms, documentID):
    tfidf = []
    # Evaluate Term Frequency
    tsvfile = open(r"C:\Users\39335\Desktop\GitHub\HW3\tsv\article_"+str(documentID)+".tsv", "r")
    reader = tsvfile.readlines()
    text = []

        # Transform the paragraph in a list with all the words
    for row in reader:
        text += row.split()

    sw = set(stopwords.words('english'))   
    # Preprocess the words

    for k in range(len(text)):
        # Remove stop words
        text[k] = text[k].lower()
        if text[k] in sw:
            text[k] = ""

        # Remove punctuation
        table = str.maketrans(dict.fromkeys(string.punctuation))
        text[k] = text[k].translate(table)

        # Stemming
        text[k] = ps.stem(text[k])

    # Remove empty elements
    text[:] = [x for x in text if x != ""]

    for term in terms:
        tf = (text.count(term) / float(len(text)))

        # Evaluate Inverse Document Frequency
        numDocumentsWithTerm = len(SearchEngine([term]))
        if numDocumentsWithTerm > 0:
            idf = 1.0 + math.log(d / numDocumentsWithTerm)
        else:
            idf = 1.0
    
        tfidf.append(tf * idf)
    return tfidf


# Returns a list with the tfIdf for each work in the query calculated in the query itself
def queryTFIDF(q):
    tfidf = []
    for i in range(len(q)):
        tfidf.append((1/len(q) * (1 + math.log(len(q)))))
    return tfidf

We now define a new Search Engine function. It will output the same dataframe as before but only with the top "k" documents with the highest cosine similarity with the query.

In [16]:
def SearchEngine2(q):
    for word in q:           # Stem the query
        word = ps.stem(word)
    df = SearchEngine1(q, ID = True)
    sim = []

    for ele in df["Document ID"]:
        qtfIdf = queryTFIDF(q)
        doctfIdf = tfIdf(q, ele)
        cosim = (np.dot(qtfIdf, doctfIdf)/(np.linalg.norm(qtfIdf)*np.linalg.norm(doctfIdf)))
        sim.append(cosim)
        
    df.insert(3, "Similarity", sim, True) 
    df = df.round({"Similarity" : 3})
    df.drop(columns="Document ID", inplace=True)
    k = 3  # How many documents we want to see
    df = df.sort_values(by='Similarity', kind='heapsort', ascending=False)  # Use heap data structure as a sorting algorithm
    return(df.head(k))

# 3) Define a new score.

We now define another Search Engine that will sort the documents that match the query in order of the budget it was spent to make it.

In [17]:
def SearchEngine3(q):
    for word in q:           # Stem the query
        word = ps.stem(word)
    df = SearchEngine1(q, ID = True)
    BoxOffice = []

    for ele in df["Document ID"]:
        filename=r'C:\Users\39335\Desktop\HW3\Data\ article_'+str(ele)+'.html'
        with io.open(filename,'r', encoding="utf-8") as f:
            s1=f.read()
            soup = BeautifulSoup(s1, 'html')
        tab=soup.find('table',class_='infobox vevent')
        box = ""
        boxdic = {}

        for r in tab.find_all('tr'):
            td1 = r.find("td")
            if td1 != None:
                s1 = td1
                p1 = r.find("th")
                if p1!=None:
                    boxdic[p1.text.strip("\n")] = s1.text.strip("\n")
        try:
            box = boxdic['Budget']
        except:
            boxdic['Budget'] = "NA"

        BoxOffice.append(box)
    df.insert(3, "Budget", BoxOffice, True) 
    df.drop(columns="Document ID", inplace=True)
    k = 3  # How many documents we want to see
    df = df.sort_values(by='Budget', kind='heapsort', ascending=False)
    return(df.head(k))

In [21]:
print("What search engine do you want to use? (1-Standard, 2-Sorted bu cosine similarity, 3-Sorted by budget")
search_engine = int(input())
print("Insert the query to search. (Insert the words separated by a space)")
q = input().split()

if search_engine == 1:
    result = SearchEngine1(q)
if search_engine == 2:
    result = SearchEngine2(q)
if search_engine == 3:
    result = SearchEngine3(q)
result

What search engine do you want to use? (1-Standard, 2-Sorted bu cosine similarity, 3-Sorted by budget
1
Insert the query to search. (Insert the words separated by a space)
mom dad


,Title,Intro,Wikipedia Url
0,Going Steady (1958 film),Going Steady is a 1958 American film about two...,https://en.wikipedia.org/wiki/Going_Steady_(19...
1,The Boy Who Cried Werewolf (1973 film),The Boy Who Cried Werewolf is a 1973 Technicol...,https://en.wikipedia.org/wiki/The_Boy_Who_Crie...
2,llflower (film),Wallflower is a 1948 American comedy film dire...,https://en.wikipedia.org/wiki/llflower_(film)
3,Bob and Sally,Bob and Sally (1948) is a movie produced in th...,https://en.wikipedia.org/wiki/Bob_and_Sally
4,Mom and D,Mom and Dad (known as The Family Story in the...,https://en.wikipedia.org/wiki/Mom_and_D
